In [7]:

from Part1 import DataPreprocessor  

# Ici, le code de Part1 ne s'est PAS lancé (grâce au if __name__ == "__main__")
# Tu peux maintenant utiliser l'outil proprement :
prepro_test = DataPreprocessor(mode='test')
prepro_test.run_pipeline()


🚀 PREPROCESSING PIPELINE - MODE: TEST

✅ Règles chargées depuis preprocessing_rules.pkl
✅ Données test chargées
✅ Sport traité (31 big clubs)
✅ Tenure et Retired traités
✅ Job traité
✅ INSEE traité (88 big cities)
✅ Densité et Revenus traités
✅ Pension et Former Jobs traités
✅ DVF traité
✅ Working hours imputé (cascade)
✅ Indépendants imputés (4797 individus)
✅ Patches finaux appliqués
✅ Catégories rares nettoyées
✅ Dataset sauvegardé: test_processed.csv

✅ PREPROCESSING TERMINÉ - 50045 lignes, 46 colonnes



In [8]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from Part1 import DataPreprocessor 

# ==============================================================================
# CONFIGURATION & PARAMETERS
# ==============================================================================
best_params = {
    'n_estimators': 150,
    'max_depth': 6,
    'learning_rate': 0.1,
    'scale_pos_weight': 1.2,
    'random_state': 42,
    'n_jobs': 2,
    'eval_metric': 'logloss',
    'tree_method': 'hist'
}

# ==============================================================================
# DATA PREPARATION
# ==============================================================================

# Run Preprocessing on Test Set
processor_test = DataPreprocessor(mode='test')
processor_test.run_pipeline()

# Load Data
df_train = pd.read_csv('train_processed.csv')
df_test = pd.read_csv('test_processed.csv')
test_ids = df_test['UNIQUE_ID']

# Target Encoding
le = LabelEncoder()
y_train = le.fit_transform(df_train['target'])

# Drop technical columns
cols_drop = ['UNIQUE_ID', 'Insee', 'work_desc', 'former_work_desc', 'target']
X_train = df_train.drop(columns=[c for c in cols_drop if c in df_train.columns])
X_test = df_test.drop(columns=[c for c in cols_drop if c in df_test.columns])

# Unified Categorical Encoding
len_train = len(X_train)
X_all = pd.concat([X_train, X_test], axis=0).reset_index(drop=True)

cats = X_all.select_dtypes(include=['object', 'category']).columns
for col in cats:
    X_all[col] = X_all[col].astype('category').cat.codes

X_all = X_all.fillna(-1)

# Resplit
X_train_final = X_all.iloc[:len_train, :]
X_test_final = X_all.iloc[len_train:, :]

# ==============================================================================
# FINAL TRAINING & PREDICTION
# ==============================================================================

# Fit on full training set
model = xgb.XGBClassifier(**best_params)
model.fit(X_train_final, y_train)

# Predict on test set
preds_encoded = model.predict(X_test_final)
preds_labels = le.inverse_transform(preds_encoded)

# Create submission dataframe
df_final = pd.DataFrame({
    'UNIQUE_ID': test_ids,
    'target': preds_labels
})

filename = "predictions.csv"
df_final.to_csv(filename, index=False)

# ==============================================================================
# FINAL CHECK
# ==============================================================================
print(f"File generated: {filename}")
print(f"Total predictions: {len(df_final)}")
print("Class distribution:")
print(df_final['target'].value_counts(normalize=True) * 100)


🚀 PREPROCESSING PIPELINE - MODE: TEST

✅ Règles chargées depuis preprocessing_rules.pkl
✅ Données test chargées
✅ Sport traité (31 big clubs)
✅ Tenure et Retired traités
✅ Job traité
✅ INSEE traité (88 big cities)
✅ Densité et Revenus traités
✅ Pension et Former Jobs traités
✅ DVF traité
✅ Working hours imputé (cascade)
✅ Indépendants imputés (4797 individus)
✅ Patches finaux appliqués
✅ Catégories rares nettoyées
✅ Dataset sauvegardé: test_processed.csv

✅ PREPROCESSING TERMINÉ - 50045 lignes, 46 colonnes



/var/folders/l0/l3b41n5n00z588bsfcg0b5040000gn/T/ipykernel_59392/2469918311.py:30: DtypeWarning: Columns (4,20,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv('train_processed.csv')
/var/folders/l0/l3b41n5n00z588bsfcg0b5040000gn/T/ipykernel_59392/2469918311.py:31: DtypeWarning: Columns (4,19,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv('test_processed.csv')


File generated: predictions.csv
Total predictions: 50045
Class distribution:
target
L    55.392147
T    44.607853
Name: proportion, dtype: float64
